In [1]:
import json

with open('/home/ubuntu/wyw/refchecker_fairness/meta_anno/data/300/gpt_score.json') as f:
    gpt_data = json.load(f) 

with open('/home/ubuntu/wyw/refchecker_fairness/meta_anno/data/300/score.json') as f:
    raw_data = json.load(f)


correct = 0
sum = 300
for i, d in enumerate(raw_data):
    d['gpt_label'] = gpt_data[i]['gpt_label']

with open('/home/ubuntu/wyw/refchecker_fairness/meta_anno/data/300/score.json', 'w') as f:
    json.dump(raw_data, f, indent=4)    

In [1]:
import json
import numpy as np

# Load the data
data_path = './data/300/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Define the range of thresholds to test
thresholds = np.logspace(-6, -2, num=10000)  # Example: from 1e-06 to 1e-03 in 100 steps

# Initialize variables to track the best threshold
best_threshold = None
best_accuracy = 0

# Loop over each threshold and evaluate performance
for threshold in thresholds:
    correct = 0
    total = 0

    for d in data:
        ref_t1 = d['ref_t1_score']
        ref_t2 = d['ref_t2_score']
        confidence = d['confidence']
        gt_labels = d['gpt_label']

        if confidence < 0.8:
            continue

        if abs(ref_t1 - ref_t2) < threshold:
            d['rc_labels'] = 2
        elif ref_t1 > ref_t2:
            d['rc_labels'] = 0
        else:
            d['rc_labels'] = 1
        
        total += 1
        if d['rc_labels'] == gt_labels:
            correct += 1
    
    # Calculate accuracy for the current threshold
    accuracy = correct / total if total > 0 else 0
    
    # Update the best threshold if the current one is better
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold

print(f'Best Threshold: {best_threshold}, Accuracy: {best_accuracy}')


Best Threshold: 0.006251433244568265, Accuracy: 0.39316239316239315


BoW result

In [3]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data
data_path = './data/300/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Initialize vectorizer
vectorizer = CountVectorizer()

# Initialize variables
threshold = 1e-06
correct = 0
total = 0

# Process each item in data
for d in data:
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    confidences = d['confidence']
    gt_labels = d['gpt_label']

    # Skip if confidence is too low
    if confidences < 0.8:
        continue

    # Vectorize the texts
    vectors = vectorizer.fit_transform([ref, text1, text2])
    
    # Compute cosine similarity
    cosine_sim_ref_text1 = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    cosine_sim_ref_text2 = cosine_similarity(vectors[0:1], vectors[2:3])[0][0]

    # Determine the label based on cosine similarity and threshold
    if abs(cosine_sim_ref_text1 - cosine_sim_ref_text2) < threshold:
        d['rc_labels'] = 2
    elif cosine_sim_ref_text1 > cosine_sim_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)


Accuracy: 0.4188034188034188


TF-IDF result

In [4]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data
data_path = './data/300/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Initialize vectorizer
vectorizer = TfidfVectorizer()

# Initialize variables
threshold = 1e-06
correct = 0
total = 0

# Process each item in data
for d in data:
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    confidences = d['confidence']
    gt_labels = d['gpt_label']

    # Skip if confidence is too low
    if confidences < 0.8:
        continue

    # Vectorize the texts
    vectors = vectorizer.fit_transform([ref, text1, text2])
    
    # Compute cosine similarity
    cosine_sim_ref_text1 = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    cosine_sim_ref_text2 = cosine_similarity(vectors[0:1], vectors[2:3])[0][0]

    # Determine the label based on cosine similarity and threshold
    if abs(cosine_sim_ref_text1 - cosine_sim_ref_text2) < threshold:
        d['rc_labels'] = 2
    elif cosine_sim_ref_text1 > cosine_sim_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)
print('Total:', total)


Accuracy: 0.41452991452991456
Total: 234


In [5]:
import json
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Load data
data_path = './data/300/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Load SimCSE model and tokenizer
model_name = "princeton-nlp/sup-simcse-roberta-large"  # Use appropriate SimCSE model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

# Initialize variables
threshold = 1e-06
correct = 0
total = 0

# Process each item in data
for d in data:
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    confidences = d['confidence']
    gt_labels = d['gpt_label']

    # Skip if confidence is too low
    if confidences < 0.8:
        continue

    # Compute embeddings
    embedding_ref = get_embedding(ref)
    embedding_text1 = get_embedding(text1)
    embedding_text2 = get_embedding(text2)

    # Compute cosine similarity
    cosine_sim_ref_text1 = cosine_similarity(embedding_ref, embedding_text1)[0][0]
    cosine_sim_ref_text2 = cosine_similarity(embedding_ref, embedding_text2)[0][0]

    # Determine the label based on cosine similarity and threshold
    if abs(cosine_sim_ref_text1 - cosine_sim_ref_text2) < threshold:
        d['rc_labels'] = 2
    elif cosine_sim_ref_text1 > cosine_sim_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)


/opt/conda/envs/refchecker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accuracy: 0.4444444444444444


In [6]:
import json
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Initialize InferSent model
from InferSent.models import InferSent
model_version = 1
MODEL_PATH = "./InferSent/encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

use_cuda = False
model = model.cuda() if use_cuda else model

W2V_PATH = './InferSent/GloVe/glove.840B.300d.txt' if model_version == 1 else 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

model.build_vocab_k_words(K=100000)

# Initialize variables
threshold = 1e-06
correct = 0
total = 0

def get_embedding(text):
    sentences = [text]
    embeddings = model.encode(sentences, bsize=128, tokenize=False)
    return embeddings[0]


# Process each item in data
for d in data:
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    confidences = d['confidence']
    gt_labels = d['gpt_label']

    # Skip if confidence is too low
    if confidences < 0.8:
        continue

    # Compute embeddings
    embedding_ref = get_embedding(ref)
    embedding_text1 = get_embedding(text1)
    embedding_text2 = get_embedding(text2)

    # Compute cosine similarity
    cosine_sim_ref_text1 = cosine_similarity([embedding_ref], [embedding_text1])[0][0]
    cosine_sim_ref_text2 = cosine_similarity([embedding_ref], [embedding_text2])[0][0]

    # Determine the label based on cosine similarity and threshold
    if abs(cosine_sim_ref_text1 - cosine_sim_ref_text2) < threshold:
        d['rc_labels'] = 2
    elif cosine_sim_ref_text1 > cosine_sim_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)


Vocab size : 100000
Accuracy: 0.4188034188034188
